In [2]:
# standard library imports
import csv
import datetime as dt
import json
import os
import statistics
import time
import tqdm

# third-party imports
import numpy as np
import pandas as pd
import requests


In [2]:
# # merge all the game data into one dataframe
# # leave this commented if you are using the provided anydeal.csv
# temp_df = pd.DataFrame()
# for i in range(0, 3000,10):
#     if i == 0:
#         temp_df = pd.read_csv(f"textfiles/game_info/last_successful_batch_{i}_game_info_anydeal.csv", index_col=0)
#     else:
#         temp_df = pd.concat([temp_df, pd.read_csv(f"textfiles/game_info/last_successful_batch_{i}_game_info_anydeal.csv", index_col=0)])
# temp_df.to_csv('anydeal.csv')

In [3]:
# # merge all the game prices into one dataframe
# temp_df = pd.DataFrame()
# for i in range(0, 3000,10):
#     if i == 0:
#         temp_df = pd.read_csv(f"textfiles/price_history/last_successful_batch_{i}_price_history.csv", index_col=0)
#     else:
#         temp_df = pd.concat([temp_df, pd.read_csv(f"textfiles/price_history/last_successful_batch_{i}_price_history.csv", index_col=0)])

# temp_df.to_csv('anydeal_prices.csv')

In [3]:
df_game_info = pd.read_csv('anydeal.csv')
df_price_history = pd.read_csv('anydeal_prices.csv')
df_price_history.rename(columns={'appid':'id'}, inplace=True)

In [4]:
df_game_info[df_game_info['id'] == '018d937f-1569-701c-bf2f-21163ded8f02']

,id,slug,title,type,mature,assets,earlyAccess,achievements,tradingCards,appid,tags,releaseDate,developers,publishers,reviews,stats,players,urls
299,018d937f-1569-701c-bf2f-21163ded8f02,total-war-shogun-2,Total War SHOGUN™ 2,game,False,{'boxart': 'https://dbxce1spal1df.cloudfront.n...,False,True,True,201270.0,"['Strategy', 'Historical', 'Turn-Based Strateg...",2011-03-14,"[{'id': 466, 'name': 'Creative Assembly'}, {'i...","[{'id': 95, 'name': 'SEGA'}, {'id': 5464, 'nam...","[{'score': 90, 'source': 'Metascore', 'count':...","{'rank': 154, 'waitlisted': 1524, 'collected':...","{'recent': 0, 'day': 1, 'week': 2, 'peak': 122}",{'game': 'https://isthereanydeal.com/game/tota...
383,018d937f-1569-701c-bf2f-21163ded8f02,total-war-shogun-2,Total War SHOGUN™ 2,game,False,{'boxart': 'https://dbxce1spal1df.cloudfront.n...,False,True,True,201270.0,"['Strategy', 'Historical', 'Turn-Based Strateg...",2011-03-14,"[{'id': 466, 'name': 'Creative Assembly'}, {'i...","[{'id': 95, 'name': 'SEGA'}, {'id': 5464, 'nam...","[{'score': 90, 'source': 'Metascore', 'count':...","{'rank': 154, 'waitlisted': 1524, 'collected':...","{'recent': 0, 'day': 1, 'week': 2, 'peak': 122}",{'game': 'https://isthereanydeal.com/game/tota...


In [5]:
# check for duplicates
df_game_info[df_game_info.duplicated('id')]

,id,slug,title,type,mature,assets,earlyAccess,achievements,tradingCards,appid,tags,releaseDate,developers,publishers,reviews,stats,players,urls
383,018d937f-1569-701c-bf2f-21163ded8f02,total-war-shogun-2,Total War SHOGUN™ 2,game,False,{'boxart': 'https://dbxce1spal1df.cloudfront.n...,False,True,True,201270.0,"['Strategy', 'Historical', 'Turn-Based Strateg...",2011-03-14,"[{'id': 466, 'name': 'Creative Assembly'}, {'i...","[{'id': 95, 'name': 'SEGA'}, {'id': 5464, 'nam...","[{'score': 90, 'source': 'Metascore', 'count':...","{'rank': 154, 'waitlisted': 1524, 'collected':...","{'recent': 0, 'day': 1, 'week': 2, 'peak': 122}",{'game': 'https://isthereanydeal.com/game/tota...
1220,018d937f-269a-71ef-ae12-7e4d194b3514,poker-night-at-the-inventory,Poker Night at the Inventory,NaN,False,[],False,False,False,NaN,[],NaN,[],[],"[{'score': 71, 'source': 'Metascore', 'count':...","{'rank': 553, 'waitlisted': 381, 'collected': ...",NaN,{'game': 'https://isthereanydeal.com/game/poke...
1670,018d937e-fdec-7040-bb13-3d96b353c3c0,wasteland-2-directors-cut,Wasteland 2: Director's Cut,game,False,{'boxart': 'https://dbxce1spal1df.cloudfront.n...,False,True,False,240760.0,"['RPG', 'Post-apocalyptic', 'Turn-Based', 'Str...",2014-09-19,"[{'id': 807, 'name': 'inXile Entertainment'}]","[{'id': 807, 'name': 'inXile Entertainment'}, ...","[{'score': 82, 'source': 'Steam', 'count': 460...","{'rank': 303, 'waitlisted': 5029, 'collected':...","{'recent': 15, 'day': 16, 'week': 19, 'peak': ...",{'game': 'https://isthereanydeal.com/game/wast...
1939,018d937f-0896-71b1-94d9-82725d1e320a,fallout-new-vegas,Fallout®: New Vegas™,game,False,{'boxart': 'https://dbxce1spal1df.cloudfront.n...,False,True,False,22380.0,"['Open World', 'RPG', 'Post-apocalyptic', 'Sin...",2010-10-19,"[{'id': 656, 'name': 'Obsidian Entertainment'}]","[{'id': 18, 'name': 'Bethesda Softworks'}, {'i...","[{'score': 96, 'source': 'Steam', 'count': 170...","{'rank': 31, 'waitlisted': 4053, 'collected': ...","{'recent': 5289, 'day': 5289, 'week': 5732, 'p...",{'game': 'https://isthereanydeal.com/game/fall...
2453,018d937f-0a68-709b-bcc7-4d842e4fe130,space-hulk-deathwing-enhanced-edition,Space Hulk: Deathwing - Enhanced Edition,game,False,{'boxart': 'https://dbxce1spal1df.cloudfront.n...,False,True,False,816090.0,"['Action', 'Warhammer 40K', 'Co-op', 'FPS', 'S...",2018-05-22,"[{'id': 1633, 'name': 'Streum On Studio'}]","[{'id': 106, 'name': 'Focus Entertainment'}, {...","[{'score': 71, 'source': 'Steam', 'count': 757...","{'rank': 3213, 'waitlisted': 2661, 'collected'...","{'recent': 53, 'day': 87, 'week': 89, 'peak': ...",{'game': 'https://isthereanydeal.com/game/spac...


In [6]:
# check for duplicates
df_price_history[df_price_history.duplicated('id')]

,id,price_history
383,018d937f-1569-701c-bf2f-21163ded8f02,"[{'timestamp': '2024-07-11T19:18:56+02:00', 's..."
1220,018d937f-269a-71ef-ae12-7e4d194b3514,"[{'timestamp': '2018-07-24T22:22:11+02:00', 's..."
1670,018d937e-fdec-7040-bb13-3d96b353c3c0,"[{'timestamp': '2024-07-11T19:19:14+02:00', 's..."
1939,018d937f-0896-71b1-94d9-82725d1e320a,"[{'timestamp': '2024-07-11T19:18:39+02:00', 's..."
2453,018d937f-0a68-709b-bcc7-4d842e4fe130,"[{'timestamp': '2024-07-25T19:16:14+02:00', 's..."


In [7]:
# remove duplicates before merging
# The duplicates are caused by the different data collection method used for steamspy and ITAD
# Steamspy used the appid from steam, which led to duplicates for different reasons, such as:
# Fallout: New Vegas has two appids on steam, one for international and one for russian version
# Total War: Shogun 2 has two appids on steam, but both lead to the same game page
# ITAD used their own unique id, which is unique for each game
# As we only use steamspy data for ranking, we will remove the duplicates from the steamspy data
# and keep the ITAD data for the duplicates

df_game_info.drop_duplicates('id', inplace=True)
df_price_history.drop_duplicates('id', inplace=True)
# merge the two dataframes

df = pd.merge(df_game_info, df_price_history, on='id', how='inner')

In [8]:
df_without_history = df[df['price_history'] == '[]']
df_without_history.to_csv('anydeal_without_history.csv')

In [9]:
# remove rows without price history
df.drop(df[df['price_history'] == '[]'].index, inplace=True)

In [10]:
df

,id,slug,title,type,mature,assets,earlyAccess,achievements,tradingCards,appid,tags,releaseDate,developers,publishers,reviews,stats,players,urls,price_history
0,018d937f-19a5-7057-bb6d-314d586e6dbc,dota-2,Dota 2,game,False,{'boxart': 'https://dbxce1spal1df.cloudfront.n...,False,False,True,570.0,"['Free to Play', 'MOBA', 'Multiplayer', 'Strat...",2013-07-09,"[{'id': 5445, 'name': 'Valve'}]","[{'id': 5445, 'name': 'Valve'}]","[{'score': 81, 'source': 'Steam', 'count': 227...","{'rank': 645, 'waitlisted': 100, 'collected': ...","{'recent': 508811, 'day': 689869, 'week': 7566...",{'game': 'https://isthereanydeal.com/game/dota...,"[{'timestamp': '2024-02-11T01:47:46+01:00', 's..."
1,018d937f-7851-7004-b780-3f657a301f9a,counter-strike-2,Counter-Strike 2,game,False,{'boxart': 'https://dbxce1spal1df.cloudfront.n...,False,False,True,730.0,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...",2012-08-21,"[{'id': 5445, 'name': 'Valve'}]","[{'id': 5445, 'name': 'Valve'}]","[{'score': 87, 'source': 'Steam', 'count': 814...","{'rank': 519, 'waitlisted': 238, 'collected': ...","{'recent': 940515, 'day': 1268934, 'week': 134...",{'game': 'https://isthereanydeal.com/game/coun...,"[{'timestamp': '2024-02-11T01:47:46+01:00', 's..."
2,018d937f-57ce-723f-b7e7-7c7b5df93471,pubg-battlegrounds,PUBG: BATTLEGROUNDS,game,False,{'boxart': 'https://dbxce1spal1df.cloudfront.n...,False,True,False,578080.0,"['Survival', 'Shooter', 'Battle Royale', 'Mult...",2017-12-21,"[{'id': 13154, 'name': 'KRAFTON, Inc.'}]","[{'id': 13154, 'name': 'KRAFTON, Inc.'}]","[{'score': 58, 'source': 'Steam', 'count': 239...","{'rank': 1682, 'waitlisted': 959, 'collected':...","{'recent': 372658, 'day': 613911, 'week': 6695...",{'game': 'https://isthereanydeal.com/game/pubg...,"[{'timestamp': '2024-02-11T03:51:23+01:00', 's..."
3,018d937f-50c1-7086-807c-e020c98c72b2,palworld,Palworld,game,False,{'boxart': 'https://dbxce1spal1df.cloudfront.n...,False,True,False,1623730.0,"['Open World', 'Multiplayer', 'Survival', 'Cre...",2024-01-19,"[{'id': 29012, 'name': 'Pocketpair'}]","[{'id': 29012, 'name': 'Pocketpair'}]","[{'score': 93, 'source': 'Steam', 'count': 268...","{'rank': 415, 'waitlisted': 10630, 'collected'...","{'recent': 38763, 'day': 46359, 'week': 59456,...",{'game': 'https://isthereanydeal.com/game/palw...,"[{'timestamp': '2024-07-18T19:16:16+02:00', 's..."
4,018d937e-fde4-72ff-a7af-45e4955a8dd6,team-fortress-2,Team Fortress 2,game,False,{'boxart': 'https://dbxce1spal1df.cloudfront.n...,False,True,True,440.0,"['Free to Play', 'Hero Shooter', 'Multiplayer'...",2007-10-10,"[{'id': 5445, 'name': 'Valve'}]","[{'id': 5445, 'name': 'Valve'}]","[{'score': 86, 'source': 'Steam', 'count': 111...","{'rank': 454, 'waitlisted': 18, 'collected': 2...","{'recent': 69106, 'day': 82557, 'week': 97322,...",{'game': 'https://isthereanydeal.com/game/team...,"[{'timestamp': '2024-02-11T01:47:46+01:00', 's..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2989,018d937e-f973-71db-988b-4b9d2c183d55,battlefleet-gothic-armada,Battlefleet Gothic: Armada,game,False,{'boxart': 'https://dbxce1spal1df.cloudfront.n...,False,True,True,363680.0,"['Strategy', 'Warhammer 40K', 'Space', 'RTS', ...",2016-04-01,"[{'id': 910, 'name': 'Tindalos Interactive'}]","[{'id': 106, 'name': 'Focus Entertainment'}, {...","[{'score': 68, 'source': 'Steam', 'count': 479...","{'rank': 3333, 'waitlisted': 2815, 'collected'...","{'recent': 26, 'day': 47, 'week': 47, 'peak': ...",{'game': 'https://isthereanydeal.com/game/batt...,"[{'timestamp': '2024-07-25T19:16:13+02:00', 's..."
2990,018d96f1-a1a9-70ec-9c8a-06ef9eb86465,occupy-mars-prologue-2020,Occupy Mars: Prologue (2020),game,False,{'boxart': 'https://dbxce1spal1df.cloudfront.n...,False,False,False,1310520.0,"['Open World Survival Craft', 'Survival', 'Ope...",2020-08-20,"[{'id': 8870, 'name': 'Pyramid Games'}]","[{'id': 10811, 'name': 'Pyramid Games S.A.'}]","[{'score': 78, 'source': 'Steam', 'count': 153...","{'rank': 24532, 'waitlisted': 99, '

In [262]:
# temp_ = df[df['id'] == '018d937f-0896-71b1-94d9-82725d1e320a'] # fallout new vegas
# temp_ = df[df['id'] == '018d937f-590c-728b-ac35-38bcff85f086'] # Elden Ring
# temp_ = df[df['id'] == '018d9584-24d6-7010-b82b-df1f0b154cc7'] # Baldur's Gate 3
# temp_ = df[df['id'] == '018d937f-0de8-7372-aa7a-fc902b95be0c'] # Outlast
# temp_ = df[df['id'] == '018d937f-3a94-7168-b777-3ac0398724cc'] # mi zhang sheng
# temp_ = df[df['id'] == '018d937f-209c-7223-8ead-671a11f25e7e'] # Dark Souls 3
# temp_ = df[df['id'] == '018d937f-089f-73ea-b66e-3167274fdfe7'] # Doom 3
# temp_ = df[df['id'] == '018d937f-19a5-7057-bb6d-314d586e6dbc'] # Dota 2
# temp_ = df[df['id'] == '018d937f-08bb-72d1-9a4a-d63dd4fa1df2'] # paladins # apparently if the game is free, its price history will lead to problems, we have to write a function to remove free games
# temp_ = df[df['id'] == '018d937f-57ce-723f-b7e7-7c7b5df93471'] # pubg


In [11]:
def handling_price_history(temp_):
    # this function will handle the price history column
    # a dataframe of the price history, based on days will be returned
    
    # # if the game is free, return an empty dataframe
    # if temp_['current_price'].iloc[0] == 0:
    #     return pd.DataFrame()
    # so I give up this idea, major reason is
    # we have games started with charge and then became free
    # for example PUBG, it was a paid game, but it became free in 2022 Jan
    # somehow, the data in isthereanydeal showed it became free in 2021 Sep


    # convert the price history from string to json
    history = temp_['price_history'].iloc[0]
    history = history.replace("'", '"')
    history = json.loads(history)

    # create the history dataframe
    history = pd.DataFrame(history)

    # convert the timestamp to datetime, keeping days only
    history['timestamp'] = pd.to_datetime(history['timestamp'],utc=True)
    history['timestamp'] = pd.to_datetime(history['timestamp'].dt.date)

    # remove the shop column, as it is the same for all rows
    history.drop(columns=['shop'], inplace=True)

    # extract the price info
    history['regular_price'] = history['deal'].apply(lambda x: x['regular']['amount'])
    history['current_price'] = history['deal'].apply(lambda x: x['price']['amount'])
    history['discount'] = history['deal'].apply(lambda x: x['cut'])

    # remove the deal column
    history.drop(columns=['deal'], inplace=True)

    # sort the dataframe by date
    history.sort_values(by='timestamp', inplace=True, ascending=True)

    return history



def daily_price_constuctor(df):
    # construct a dataframe with the every day price history based on a given date range
    # will be used to calculate the average price for each game 
    # as average price will be weighted by the number of days

    start_date = df.iloc[0]['timestamp']
    end_date = df.iloc[-1]['timestamp']
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    price_history = pd.DataFrame()
    price_history['timestamp'] = date_range
    price_history['timestamp'] = pd.to_datetime(price_history['timestamp'])
    price_history = price_history.merge(df, how='left', on='timestamp')
    price_history['timestamp'] = price_history['timestamp'].dt.date
    price_history['regular_price'].ffill(inplace=True)
    price_history['current_price'].ffill(inplace=True)
    price_history['discount'].ffill(inplace=True)

    return price_history

# test_function_var = handling_price_history(temp_)

# print(test_function_var.to_string())

# print(daily_price_constuctor(test_function_var).to_string())


In [12]:

price_items = {
    'first_available_price': '', # the first price of the game
    'regular_price_reduced': '', # boolean, true if the regular price has been reduced
    'price_reduced_date': '', # null if regular_price_reduced is false
    'first_available_date': '', # date of the first available price
    'last_available_date': '', # date of the last available price
    'historical_low': '', # the lowest price of the game
    'historical_low_date': '', # the date of the lowest price
    'historical_high_discount': '', # the highest discount of the game
    'historical_high_discount_date': '', # the date of the highest discount
    'first_discount': '', # the first discount of the game
    'first_discount_date': '', # the date of the first discount
    'last_discount': '', # the last discount of the game
    'last_discount_date': '', # the date of the last discount
    'discount_distribution': '', # the distribution of discounts based on months


    

    # the following modules require the daily price history
    'average_duration_between_discounts_12months': '', # the average duration between discounts in the last 12 months
    'average_duration_of_discounts_12months': '', # the average duration of discounts in the last 12 months
    'avg_price': '', # the average price of the game 
    'avg_discount': '', # the average discount of the game, calculated as the average of the discount percentage
    # time series analysis items
    'next_discount': '',
    # price decay is the average second derivative of a moving average of the price
    # we take 12 months of data, and calculate the moving average of the price
    # then we calculate the second derivative of the moving average
    # finally we average the second derivative over the 12 months
    'price_decay_rate': ''

}

price_items_names = list(price_items.keys())

def calculate_price_items(temp_):
    temp_ = temp_.to_frame().T
    print('now processing:', temp_['title'].iloc[0])
    history = handling_price_history(temp_)


    # calculate the items

    first_available_price = history['regular_price'].iloc[0]
    if first_available_price == 0:
        # if the game started as free, return an empty dictionary
        return [None for i in range(len(price_items_names))]
    
    # if the regular price has been reduced
    if history['regular_price'].unique().shape[0] > 1:
        prices = history['regular_price'].unique()
        # print(prices)
        regular_price_reduced = True
        # find the date when the price was reduced
        price_reduced_date = history[history['regular_price'] == prices[1]]['timestamp'].iloc[0]
    else:
        regular_price_reduced = False
        price_reduced_date = None

    first_available_date = history['timestamp'].iloc[0]

    last_available_date = history['timestamp'].iloc[-1]

    historical_low = history['current_price'].min()

    if historical_low == first_available_price:
        historical_low_date = None
    else:
        historical_low_date = history[history['current_price'] == historical_low]['timestamp'].iloc[0]

    historical_high_discount = history['discount'].max()
    if historical_high_discount == 0:
        historical_high_discount_date = None
        first_discount = None
        first_discount_date = None
        last_discount = None
        last_discount_date = None
        discount_distribution = None
    else:
        historical_high_discount_date = history[history['discount'] == historical_high_discount]['timestamp'].iloc[0]
        first_discount = history[history['discount'] != 0]['discount'].iloc[0]
        first_discount_date = history[history['discount'] != 0]['timestamp'].iloc[0]
        last_discount = history[history['discount'] != 0]['discount'].iloc[-1]
        last_discount_date = history[history['discount'] != 0]['timestamp'].iloc[-1]
        discount_distribution = history[history['discount'] != 0].groupby(history['timestamp'].dt.month)['discount'].count()
    # print(discount_distribution.to_string())
    # calculate the average duration between discounts

    daily_price = daily_price_constuctor(history)

    average_duration_between_discounts_12months = None

    average_duration_of_discounts_12months = None

    avg_price = None

    avg_discount = None

    next_discount = None

    price_decay_rate = None



    price_items_inside = {
        'first_available_price': first_available_price,
        'regular_price_reduced': regular_price_reduced, 
        'price_reduced_date': price_reduced_date, 
        'first_available_date': first_available_date, 
        'last_available_date': last_available_date, 
        'historical_low': historical_low, 
        'historical_low_date': historical_low_date,
        'historical_high_discount': historical_high_discount,
        'historical_high_discount_date': historical_high_discount_date,
        'first_discount': first_discount,
        'first_discount_date': first_discount_date,
        'last_discount': last_discount,
        'last_discount_date': last_discount_date,
        'discount_distribution': discount_distribution,
        'average_duration_between_discounts_12months': average_duration_between_discounts_12months,
        'average_duration_of_discounts_12months': average_duration_of_discounts_12months,
        'avg_price': avg_price,
        'avg_discount': avg_discount,
        'next_discount': next_discount,
        'price_decay_rate': price_decay_rate

    }

    price_items_values = list(price_items_inside.values())
    # price_items_values = [i for i in range(len(price_items_inside.keys()))]

    # print(pd.Series(price_items_values, index=price_items_names))
    # print(history.to_string())
    # print(price_items_values)
    return price_items_values

# calculate_price_items(temp_)

In [13]:
# apply the function to the dataframe
df[price_items_names] = df.apply(calculate_price_items, axis=1, result_type='expand')
df.drop(columns=['price_history','assets'], inplace=True)
df.to_csv('price_items.csv')
# test_head = df.head(10)
# test_head[price_items_names] = test_head.apply(calculate_price_items, axis=1, result_type='expand')
# test_head.drop(columns=['price_history','assets','developers','reviews','stats','players'], inplace=True)
# print(test_head.to_string())

now processing: Dota 2
now processing: Counter-Strike 2
now processing: PUBG: BATTLEGROUNDS
now processing: Palworld
now processing: Team Fortress 2
now processing: Apex Legends™
now processing: New World
now processing: Call of Duty®
now processing: Grand Theft Auto V
now processing: Left 4 Dead 2
now processing: Rust
now processing: Lost Ark
now processing: Unturned
now processing: Warframe
now processing: HELLDIVERS 2
now processing: War Thunder
now processing: Elden Ring
now processing: Terraria
now processing: Brawlhalla
now processing: Garry's Mod
now processing: Baldur's Gate 3
now processing: Tom Clancy's Rainbow Six® Siege
now processing: Destiny 2
now processing: Wallpaper Engine
now processing: Half-Life 2: Lost Coast
now processing: ARK: Survival Evolved
now processing: Valheim
now processing: Cyberpunk 2077
now processing: PAYDAY 2
now processing: Grand Theft Auto IV: Complete Edition
now processing: The Forest
now processing: Path of Exile
now processing: Don't Starve Tog

/var/folders/7m/bpynmdb95w587ztb6nlzcgg80000gn/T/ipykernel_16409/684933141.py:23: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  history['timestamp'] = pd.to_datetime(history['timestamp'],utc=True)


now processing: Neon Abyss
now processing: Football Manager 2023
now processing: Creative Destruction
now processing: Chivalry 2
now processing: Soundpad
now processing: Kingdom: Two Crowns
now processing: Serious Sam 2
now processing: Knight Online
now processing: Naruto Shippuden: Ultimate Ninja Storm 4
now processing: Resident Evil Revelations 2 / Biohazard Revelations 2 Deluxe Edition
now processing: Warhammer 40,000: Rogue Trader
now processing: A Way Out
now processing: Ghost of Tsushima DIRECTOR'S CUT
now processing: Resident Evil® 5
now processing: Lords Of The Fallen
now processing: Naruto to Boruto Shinobi Striker
now processing: BioShock 2 Remastered
now processing: Shadowverse CCG
now processing: OUTRIDERS
now processing: Ghostrunner
now processing: Alien Swarm: Reactive Drop
now processing: Tropico 4
now processing: State of Decay 2: Juggernaut Edition
now processing: Vindictus
now processing: Back to Bed
now processing: RuneScape
now processing: KARDS - The WW2 Card Game


/var/folders/7m/bpynmdb95w587ztb6nlzcgg80000gn/T/ipykernel_16409/684933141.py:23: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  history['timestamp'] = pd.to_datetime(history['timestamp'],utc=True)


now processing: Tom Clancy's Rainbow Six Vegas 2
now processing: The Messenger
now processing: Totally Reliable Delivery Service Beta
now processing: Bloodline Champions
now processing: THE KING OF FIGHTERS XIII STEAM EDITION
now processing: Animal Jam
now processing: Curious Expedition
now processing: Slay the Princess
now processing: Receiver
now processing: Barro
now processing: Minimum
now processing: Nickelodeon All-Star Brawl
now processing: Secret Of Magia
now processing: The Case of the Golden Idol
now processing: Velvet Sundown
now processing: Manual Samuel - Anniversary Edition
now processing: Halo Wars: Definitive Edition
now processing: Divinity II Developer's Cut
now processing: QUAKE III Arena
now processing: Pixel Puzzles Ultimate Jigsaw
now processing: Rakion Chaos Force
now processing: Far Cry 6
now processing: Kingdom Wars
now processing: VITATIO 2
now processing: F1® 2019 Anniversary Edition
now processing: The Coin Game
now processing: Chaos on Deponia
now processin

In [14]:
# 12 months average price
# def monthly_avg_price(year, month, df):
#     monthprice = df[(df['timestamp'].dt.year == year) & (df['timestamp'].dt.month == month)]
#     if monthprice.empty:
#         return None
#     else:
#         current_index = monthprice.index[0]
#         start_price = df['current_price'].iloc[current_index-1]
#         return start_price

# monthly_avg_price(2021, 11, history)


NameError: name 'history' is not defined

In [ ]:

price_items = {
    'first_available_price': first_available_price,
    'regular_price_reduced': '', # boolean
    'price_reduced_date': '', # null if regular_price_reduced is false
    # 'first_available_date': '', # we already have it in the game info
    'historical_low': '',
    'historical_low_date': '',
    'historical_high_discount': '',
    'historical_high_discount_date': '',
    'first_discount': '',
    'first_discount_date': '',
    'last_discount': '',
    'last_discount_date': '',
    'average_duration_between_discounts_12months': '',
    'average_duration_of_discounts_12months': '',
    'avg_price': '',
    'avg_discount': '',

    

    # testing modules
    # time series analysis items
    'next_discount': '',
    # price decay is the average second derivative of a moving average of the price
    # we take 12 months of data, and calculate the moving average of the price
    # then we calculate the second derivative of the moving average
    # finally we average the second derivative over the 12 months
    'price_decay_rate': '', 

}

# get price history for a game


# DEV NOTEs

In [ ]:
############# dev notes ####################



# Solution 02.
# Use result_type ='expand' when applying.

# def myfunc2(args):
#     e = args[0] + 2*args[1]
#     f = args[1]*args[2] + 1
#     g = args[2] + args[0] * args[1]
#     return [e, f, g]

# df[['e', 'f', 'g']] = df.apply(myfunc2, axis=1, result_type='expand')
# #                          x         y         z         e         f         g
# # ts                                                                          
# # 2014-05-15 10:38  0.120117  0.987305  0.116211  2.094727  1.114736  0.234803
# # 2014-05-15 10:39  0.117188  0.984375  0.122070  2.085938  1.120163  0.237427
# # 2014-05-15 10:40  0.119141  0.987305  0.119141  2.093751  1.117629  0.236770
# # 2014-05-15 10:41  0.116211  0.984375  0.120117  2.084961  1.118240  0.234512
# # 2014-05-15 10:42  0.119141  0.983398  0.118164  2.085937  1.116202  0.235327

# t2 = timeit.timeit(
#     "df.apply(myfunc2, axis=1, result_type='expand')",
#     globals=dict(df=df, myfunc2=myfunc2), number=10000)
# print(round(t2, 3), 'seconds')
# 9.907 seconds


## to do
- ~~I didn't take increasing prices into consideration, which shall be a rare case. But it should be tested.~~ After EDA, we noticed that many games choose to do a first week discount, which will lead to a increase in price.

### idea: 等等党模拟器
- 从数据中随机抽样，模拟等等党等多久能介到第一次打折？

### idea: 何时打折.jpg
- 调用API，选取所有公司，基于数据给出合理的打折预估